In [1]:
from fyers_apiv3 import fyersModel
from fyers_apiv3.FyersWebsocket import data_ws
import pandas as pd
import time
from datetime import date
from datetime import datetime
import pytz

# Replace these values with your actual API credentials
client_id = "PP860PBEU0-100"
secret_key = "JC501DPXH7"
redirect_uri = "https://google.com"
response_type = "code"  
state = "sample_state"
session = fyersModel.SessionModel(
    client_id=client_id,
    secret_key=secret_key,
    redirect_uri=redirect_uri,
    response_type=response_type
)
response = session.generate_authcode()
print(response)

https://api-t1.fyers.in/api/v3/generate-authcode?client_id=PP860PBEU0-100&redirect_uri=https%3A%2F%2Fgoogle.com&response_type=code&state=None


In [2]:
auth_code = "eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJhcGkubG9naW4uZnllcnMuaW4iLCJpYXQiOjE3MDI5Mjg0MTUsImV4cCI6MTcwMjk1ODQxNSwibmJmIjoxNzAyOTI3ODE1LCJhdWQiOiJbXCJ4OjBcIiwgXCJ4OjFcIiwgXCJ4OjJcIiwgXCJkOjFcIiwgXCJkOjJcIiwgXCJ4OjFcIiwgXCJ4OjBcIl0iLCJzdWIiOiJhdXRoX2NvZGUiLCJkaXNwbGF5X25hbWUiOiJYUzk4MzY4Iiwib21zIjoiSzEiLCJoc21fa2V5IjoiMzhlNjgzNTVmMGQ3NTUyMTYyZDJhN2RlMzBkYTJjZjI4YzhjYmRjNjEzNWVkYmEyYjJkNTc2MjciLCJub25jZSI6IiIsImFwcF9pZCI6IlBQODYwUEJFVTAiLCJ1dWlkIjoiNjMxY2JhNGNlZTFiNDViZmIzNjI0YWEyZWI3YmZlODUiLCJpcEFkZHIiOiIwLjAuMC4wIiwic2NvcGUiOiIifQ.GkY6Q_0JeEZgXdrAvCsu5LGMVLgMMK1gFu-6GWsaoH4"

In [3]:
grant_type = "authorization_code"
session = fyersModel.SessionModel(
    client_id=client_id,
    secret_key=secret_key, 
    redirect_uri=redirect_uri, 
    response_type=response_type, 
    grant_type=grant_type
)
session.set_token(auth_code)
response = session.generate_token()
print(response)

{'s': 'ok', 'code': 200, 'message': '', 'access_token': 'eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJhcGkuZnllcnMuaW4iLCJpYXQiOjE3MDI5Mjg0NDIsImV4cCI6MTcwMjk0NTg0MiwibmJmIjoxNzAyOTI4NDQyLCJhdWQiOlsieDowIiwieDoxIiwieDoyIiwiZDoxIiwiZDoyIiwieDoxIiwieDowIl0sInN1YiI6ImFjY2Vzc190b2tlbiIsImF0X2hhc2giOiJnQUFBQUFCbGdLQTYxVmp6aFRTSEFRNl90eDMwekY0QWhUQVpUQlV5a3ZuZ2lFNVcxR014ZDhRUnRBUTRKbFBfQmhPdFl3QkdEclVaZ1htSENqRmZjaDkyN2o5SFE2VmNULUZKRkxQM01Ucm44bU9HNGY5aFRHbz0iLCJkaXNwbGF5X25hbWUiOiJTQU5TS0FSIFNSSVZBU1RBVkEiLCJvbXMiOiJLMSIsImhzbV9rZXkiOiIzOGU2ODM1NWYwZDc1NTIxNjJkMmE3ZGUzMGRhMmNmMjhjOGNiZGM2MTM1ZWRiYTJiMmQ1NzYyNyIsImZ5X2lkIjoiWFM5ODM2OCIsImFwcFR5cGUiOjEwMCwicG9hX2ZsYWciOiJOIn0.8tHIUetPCs2peiCT60uSlSOtfv1A214B0diIR8ML1Qk', 'refresh_token': 'eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJhcGkuZnllcnMuaW4iLCJpYXQiOjE3MDI5Mjg0NDIsImV4cCI6MTcwNDE1NTQ0MiwibmJmIjoxNzAyOTI4NDQyLCJhdWQiOlsieDowIiwieDoxIiwieDoyIiwiZDoxIiwiZDoyIiwieDoxIiwieDowIl0sInN1YiI6InJlZnJlc2hfdG9rZW4iLCJhdF9oYXNoIjoiZ0FBQUFBQ

In [4]:
df=pd.DataFrame()
df=response
tk=df['access_token']
print(tk)

eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJhcGkuZnllcnMuaW4iLCJpYXQiOjE3MDI5Mjg0NDIsImV4cCI6MTcwMjk0NTg0MiwibmJmIjoxNzAyOTI4NDQyLCJhdWQiOlsieDowIiwieDoxIiwieDoyIiwiZDoxIiwiZDoyIiwieDoxIiwieDowIl0sInN1YiI6ImFjY2Vzc190b2tlbiIsImF0X2hhc2giOiJnQUFBQUFCbGdLQTYxVmp6aFRTSEFRNl90eDMwekY0QWhUQVpUQlV5a3ZuZ2lFNVcxR014ZDhRUnRBUTRKbFBfQmhPdFl3QkdEclVaZ1htSENqRmZjaDkyN2o5SFE2VmNULUZKRkxQM01Ucm44bU9HNGY5aFRHbz0iLCJkaXNwbGF5X25hbWUiOiJTQU5TS0FSIFNSSVZBU1RBVkEiLCJvbXMiOiJLMSIsImhzbV9rZXkiOiIzOGU2ODM1NWYwZDc1NTIxNjJkMmE3ZGUzMGRhMmNmMjhjOGNiZGM2MTM1ZWRiYTJiMmQ1NzYyNyIsImZ5X2lkIjoiWFM5ODM2OCIsImFwcFR5cGUiOjEwMCwicG9hX2ZsYWciOiJOIn0.8tHIUetPCs2peiCT60uSlSOtfv1A214B0diIR8ML1Qk


In [5]:
fyers = fyersModel.FyersModel(client_id=client_id, is_async=False, token=tk, log_path="")

In [6]:
def signal_generator(df):
    Open = df.open.iloc[-1]
    Close = df.close.iloc[-1]
    previous_Open = df.open.iloc[-2]
    previous_Close = df.close.iloc[-2]
    
    # red candle
    if (Open>Close):
        return 1

    # green candle
    elif (Open<Close):
        return 2
    
    # No clear pattern
    else:
        return 0


In [7]:
call=input("Enter the symbol for call : ")

Enter the symbol for call : NSE:BANKNIFTY23D2047800CE


In [8]:
put=input("Enter the symbol for put : ")

Enter the symbol for put : NSE:BANKNIFTY23D2047100PE


In [9]:
calltight=int(input("Enter qty for tight lots for call option:  "))

Enter qty for tight lots for call option:  15


In [10]:
callspare=int(input("Enter qty for spare lots for call option:  "))

Enter qty for spare lots for call option:  15


In [11]:
puttight=int(input("Enter qty for tight lots for put option:  "))

Enter qty for tight lots for put option:  15


In [12]:
putspare=int(input("Enter qty for spare lots for put option:  "))

Enter qty for spare lots for put option:  15


In [13]:
reso=input("Enter the resolution : ")

Enter the resolution : 5


In [14]:
goodpairCE = pd.DataFrame(columns=['Date', 'Time', 'HC', 'LC', 'ET', 'ES', 'orderIDT', 'orderIDS', 'Target', 'SL'])
global goodpairCE

In [19]:
def get_datacall():
    data = {
    "symbol":call,
    "resolution":reso,
    "date_format":"1",
    "range_from":date.today(),
    "range_to":date.today(),
    "cont_flag":"1"
    }

    hdata=fyers.history(data=data)
    df = pd.DataFrame.from_dict(hdata['candles'])
    cols = ['datetime','open','high','low','close','volume']
    df.columns=cols
    df['datetime']= pd.to_datetime(df['datetime'],unit="s")
    df['datetime'] = df['datetime'].dt.tz_localize('utc').dt.tz_convert('Asia/Kolkata')
    df['datetime'] = df['datetime'].dt.tz_localize(None)
    df['date'] = df['datetime'].dt.date
    df['time'] = df['datetime'].dt.time
    
    signal = []
    signal.append(0)
    for i in range(1,len(df)):
        dataF = df[i-1:i+1]
        signal.append(signal_generator(dataF))
    df["signal"] = signal
    
    h=0
    l=0
    t=0
    signalfirst = df.signal.iloc[-1]
    signalsecond = df.signal.iloc[-2]
    presenthigh = df.high.iloc[-1]
    previoushigh = df.high.iloc[-2]
    presentlow = df.low.iloc[-1]
    previouslow = df.low.iloc[-2]
    previoustime = df.time.iloc[-2]
    presenttime = df.time.iloc[-1]
    previousdate = df.date.iloc[-2]
    presentdate = df.date.iloc[-1]
    
    if(signalfirst!=signalsecond):
        if(previoushigh>presenthigh and presentlow<previouslow):
            h = previoushigh
            l = presentlow
            t=1
        elif(previoushigh<presenthigh and presentlow>previouslow):
            h = presenthigh
            l = previouslow
            t=1
        elif(presentlow<previouslow and presenthigh>previoushigh and 0<=(presenthigh-previoushigh)<=1):
            h=presenthigh
            l=presentlow
            t=1
        elif(previouslow<presentlow and previoushigh>presenthigh and 0<=(previoushigh-presenthigh)<=1):
            h=previoushigh
            l=previouslow
            t=1
    if(abs(h-l)>=0 and abs(h-l)<=30 and t==1):
        goodpairCE.loc[len(goodpairCE)] = [previousdate, previoustime, h, l, 0, 0, 0, 0, 0, 0]

In [20]:
get_datacall()
goodpairCE

,Date,Time,HC,LC,ET,ES,orderIDT,orderIDS,Target,SL


In [21]:
goodpairPE = pd.DataFrame(columns=['Date', 'Time', 'HP', 'LP', 'ET', 'ES', 'orderIDT', 'orderIDS', 'Target', 'SL'])
global goodpairPE

In [32]:
def get_dataput():
    data = {
    "symbol":put,
    "resolution":reso,
    "date_format":"1",
    "range_from":date.today(),
    "range_to":date.today(),
    "cont_flag":"1"
    }

    hdata=fyers.history(data=data)
    df = pd.DataFrame.from_dict(hdata['candles'])
    cols = ['datetime','open','high','low','close','volume']
    df.columns=cols
    df['datetime']= pd.to_datetime(df['datetime'],unit="s")
    df['datetime'] = df['datetime'].dt.tz_localize('utc').dt.tz_convert('Asia/Kolkata')
    df['datetime'] = df['datetime'].dt.tz_localize(None)
    df['date'] = df['datetime'].dt.date
    df['time'] = df['datetime'].dt.time
    
    signal = []
    signal.append(0)
    for i in range(1,len(df)):
        dataF = df[i-1:i+1]
        signal.append(signal_generator(dataF))
    df["signal"] = signal
    
    h=0
    l=0
    t=0
    signalfirst = df.signal.iloc[-1]
    signalsecond = df.signal.iloc[-2]
    presenthigh = df.high.iloc[-1]
    previoushigh = df.high.iloc[-2]
    presentlow = df.low.iloc[-1]
    previouslow = df.low.iloc[-2]
    previoustime = df.time.iloc[-2]
    presenttime = df.time.iloc[-1]
    previousdate = df.date.iloc[-2]
    presentdate = df.date.iloc[-1]
    
    if(signalfirst!=signalsecond):
        if(previoushigh>presenthigh and presentlow<previouslow):
            h = previoushigh
            l = presentlow
            t=1
        elif(previoushigh<presenthigh and presentlow>previouslow):
            h = presenthigh
            l = previouslow
            t=1
        elif(presentlow<previouslow and presenthigh>previoushigh and 0<=(presenthigh-previoushigh)<=1):
            h=presenthigh
            l=presentlow
            t=1
        elif(previouslow<presentlow and previoushigh>presenthigh and 0<=(previoushigh-presenthigh)<=1):
                h=previoushigh
                l=previouslow
                t=1
    if(abs(h-l)>=0 and abs(h-l)<=30 and t==1):
        goodpairPE.loc[len(goodpairPE)] = [previousdate, previoustime, h, l, 0, 0, 0, 0, 0, 0]

In [33]:
get_dataput()
goodpairPE

,Date,Time,HP,LP,ET,ES,orderIDT,orderIDS,Target,SL


In [34]:
def onmessage(message):
    timezone = pytz.timezone('Asia/Kolkata')
    t= datetime.now(timezone)
    chr = t.hour
    cmin = t.minute
    csec = t.second
    if (int(cmin) % int(reso) == 0 and int(csec)<3):
        get_datacall()
        get_dataput()
    print("Response : ", message)
    Funds = fyers.funds()
    if (chr==15 and cmin==15 and csec==0):
        closedata = {}
        closeresponse = fyers.exit_positions(data=closedata)
        print(closeresponse) 
    if (Funds['fund_limit'][0]['equityAmount']>15000 and len(goodpairCE)>=1 and goodpairCE.Date.iloc[-1]==date.today() and goodpairCE.ET.iloc[-1]==0 and message['symbol']==call and message['ltp']>=goodpairCE.HC.iloc[-1]):
        cancelpending = {}
        Response = fyers.cancel_order(data=cancelpending)
        data_ct = {
            "symbol":call,
            "qty":calltight,
            "type":4,
            "side":1,
            "productType":"BO",
            "limitPrice":goodpairCE.HC.iloc[-1],
            "stopPrice":goodpairCE.HC.iloc[-1],
            "validity":"DAY",
            "disclosedQty":0,
            "offlineOrder":False,
            "stopLoss":goodpairCE.HC.iloc[-1]-goodpairCE.LC.iloc[-1],
            "takeProfit":15
        }
        response_ct = fyers.place_order(data=data_ct)
        print(response_ct)
        if response_ct['s']=="ok":
            print(f"entry tight slot for call {call}")
            goodpairCE.ET.iloc[-1]=1
            goodpairCE.orderIDT.iloc[-1]=response_ct["id"]
        
    if (Funds['fund_limit'][0]['equityAmount'] and len(goodpairPE)>=1 and goodpairPE.Date.iloc[-1]==date.today() and goodpairPE.ET.iloc[-1]==0 and message['symbol']==put and message['ltp']>=goodpairPE.HP.iloc[-1]):
        cancelpending = {}
        Response = fyers.cancel_order(data=cancelpending)
        data_pt = {
            "symbol":put,
            "qty":puttight,
            "type":4,
            "side":1,
            "productType":"BO",
            "limitPrice":goodpairPE.HP.iloc[-1],
            "stopPrice":goodpairPE.HP.iloc[-1],
            "validity":"DAY",
            "disclosedQty":0,
            "offlineOrder":False,
            "stopLoss":goodpairPE.HP.iloc[-1]-goodpairPE.LP.iloc[-1],
            "takeProfit":15
        }
        response_pt = fyers.place_order(data=data_pt)
        print(response_pt)
        if response_pt['s']=="ok":
            print(f"entry tight slot for put {put}")
            goodpairPE.ET.iloc[-1]=1
            goodpairPE.orderIDT.iloc[-1]=response_pt['id']
            
    if (Funds['fund_limit'][0]['equityAmount'] and len(goodpairCE)>=1 and goodpairCE.Date.iloc[-1]==date.today() and goodpairCE.ES.iloc[-1]==0 and message['symbol']==call and message['ltp']>=goodpairCE.HC.iloc[-1]):
        cancelpending = {}
        Response = fyers.cancel_order(data=cancelpending)
        data_cs = {
            "symbol":call,
            "qty":callspare,
            "type":4,
            "side":1,
            "productType":"CO",
            "limitPrice":goodpairCE.HC.iloc[-1],
            "stopPrice":goodpairCE.HC.iloc[-1],
            "validity":"DAY",
            "disclosedQty":0,
            "offlineOrder":False,
            "stopLoss":goodpairCE.HC.iloc[-1]-goodpairCE.LC.iloc[-1],
            "takeProfit":0
        }
        response_cs = fyers.place_order(data=data_cs)
        print(response_cs)
        if response_cs['s']=='ok':
            print(f"entry spare slot for call {call}")
            goodpairCE.orderIDS.iloc[-1]=response_cs['id']
            goodpairCE.Target.iloc[-1]=goodpairCE.HC.iloc[-1]+15
            goodpairCE.SL.iloc[-1]=goodpairCE.LC.iloc[-1]
            goodpairCE.ES.iloc[-1]=1
            
    for i in range(len(goodpairCE)):
        if(goodpairCE.ES.iloc[i]==1 and message['symbol']==call and message['ltp']>=goodpairCE.Target.iloc[i]):
            goodpairCE.Target.iloc[i]+=15
            goodpairCE.SL.iloc[i]=goodpairCE.Target.iloc[i]-30
            orderIdC=goodpairCE.orderIDS.iloc[i]
            data_mcs = {
                "id":orderIdC, 
                "type":1, 
                "limitPrice":goodpairCE.Target.iloc[i]-15,
                "stopPrice":goodpairCE.Target.iloc[i]-15,
                "qty":callspare
            }
            response_mcs = fyers.modify_order(data=data_mcs)
            if response_mcs['s']=='ok':
                print(f"spare slot modified for call={call}, orderID={goodpairCE.orderIDS.iloc[i]}, target={goodpairCE.Target.iloc[i]}, sl={goodpairCE.SL.iloc[i]}")
        if(goodpairCE.ES.iloc[i]==1 and message['symbol']==call and message['ltp']<=goodpairCE.SL.iloc[i]):
            exit_cs = {"id":goodpairCE.orderIDS.iloc[i]}
            exit_response_cs = fyers.exit_positions(data=exit_cs)
            print(f"EXIT {goodpairCE.orderIDS.iloc[i]}")
            
    if (Funds['fund_limit'][0]['equityAmount'] and len(goodpairPE)>=1 and goodpairPE.Date.iloc[-1]==date.today() and goodpairPE.ES.iloc[-1]==0 and message['symbol']==put and message['ltp']>=goodpairPE.HP.iloc[-1]):
        cancelpending = {}
        Response = fyers.cancel_order(data=cancelpending)
        data_ps = {
            "symbol":put,
            "qty":putspare,
            "type":4,
            "side":1,
            "productType":"CO",
            "limitPrice":goodpairPE.HP.iloc[-1],
            "stopPrice":goodpairPE.HP.iloc[-1],
            "validity":"DAY",
            "disclosedQty":0,
            "offlineOrder":False,
            "stopLoss":goodpairPE.HP.iloc[-1]-goodpairPE.LP.iloc[-1],
            "takeProfit":0
        }
        response_ps = fyers.place_order(data=data_ps)
        print(response_ps)
        if response_ps['s']=='ok':
            print(f"entry spare slot for put {put}")
            goodpairPE.orderIDS.iloc[-1]=response_ps['id']
            goodpairPE.Target.iloc[-1]=goodpairPE.HP.iloc[-1]+15
            goodpairPE.SL.iloc[-1]=goodpairPE.LP.iloc[-1]
            goodpairPE.ES.iloc[-1]=1
            
    for i in range(len(goodpairPE)):
        if(goodpairPE.ES.iloc[-1]==1 and message['symbol']==put and message['ltp']>=goodpairPE.Target.iloc[i]):
            goodpairPE.Target.iloc[i]+=15
            goodpairPE.SL.iloc[i]=goodpairPE.Target.iloc[i]-30
            orderIdP=goodpairPE.orderIDS.iloc[i]
            data_mps = {
                "id":orderIdP, 
                "type":1, 
                "stopPrice": goodpairPE.Target.iloc[i]-15,
                "qty":putspare
            }
            response_mps = fyers.modify_order(data=data_mps)
            if response_mps['s']=='ok':
                print(f"spare slot modified for put={put}, orderID={goodpairPE.orderIDS.iloc[i]}, target={goodpairPE.Target.iloc[i]}, sl={goodpairPE.SL.iloc[i]}")
        if(goodpairPE.ES.iloc[i]==1 and message['symbol']==put and message['ltp']<=goodpairPE.SL.iloc[i]):
            exit_ps = {"id":goodpairPE.orderIDS.iloc[i]}
            exit_response_ps = fyers.exit_positions(data=exit_ps)
            print(f"EXIT {goodpairPE.orderIDS.iloc[i]}")
                
def onerror(message):
    print("Error:", message)

def onclose(message):
    print("Connection closed:", message)

def onopen():
    data_type = "SymbolUpdate"
    symbols = [call,put]
    ofyers.subscribe(symbols=symbols, data_type=data_type)
    ofyers.keep_running()
access_token = client_id + ":" + tk

ofyers = data_ws.FyersDataSocket(
    access_token=access_token,
    log_path="",                     
    litemode=False,
    write_to_file=False,              
    reconnect=True,                 
    on_connect=onopen,               
    on_close=onclose,                
    on_error=onerror,               
    on_message=onmessage             
)
ofyers.connect()

Response :  {'type': 'cn', 'code': 200, 'message': 'Authentication done', 's': 'ok'}
Response :  {'type': 'ful', 'code': 200, 'message': 'Full Mode On', 's': 'ok'}
Response :  {'type': 'sub', 'code': 200, 'message': 'Subscribed', 's': 'ok'}
Response :  {'ltp': 205.15, 'vol_traded_today': 58497720, 'last_traded_time': 1702893599, 'exch_feed_time': 1702893600, 'bid_size': 255, 'ask_size': 390, 'bid_price': 197.2, 'ask_price': 204.8, 'last_traded_qty': 15, 'tot_buy_qty': 80070, 'tot_sell_qty': 244260, 'avg_trade_price': 301.53, 'low_price': 193.35, 'high_price': 427.85, 'lower_ckt': 0, 'upper_ckt': 0, 'open_price': 424.95, 'prev_close_price': 512.35, 'type': 'sf', 'symbol': 'NSE:BANKNIFTY23D2047800CE', 'ch': -307.2, 'chp': -59.959}
Response :  {'ltp': 25.3, 'vol_traded_today': 12376560, 'last_traded_time': 1702893599, 'exch_feed_time': 1702893600, 'bid_size': 3465, 'ask_size': 900, 'bid_price': 25.3, 'ask_price': 25.65, 'last_traded_qty': 15, 'tot_buy_qty': 76755, 'tot_sell_qty': 71865, '